# **1.0 Data Scrapping**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import re

In [2]:

url = 'https://www2.hm.com/en_us/men/products/jeans.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)

#Encontrando o html dos produtos
soup = BeautifulSoup(page.text, 'html.parser')
products = soup.find('ul', class_ = 'products-listing small')
products_list = products.find_all('article', class_= 'hm-product-item')


#Product id
product_id = [p.get('data-articlecode') for p in products_list]


#Product Category
product_category = [p.get('data-category') for p in products_list]

#Product name
products_list = products.find_all('a', class_= 'link')
product_name = [p.get_text() for p in products_list]

#Price
price_list = products.find_all('span', class_= 'price regular')
product_price = [ p.get_text() for p in price_list]
product_price

data = pd.DataFrame([product_id, product_category, product_name,  product_price]).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

# **2.0 Data Collection by Product**

In [47]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
# empty dataframe
aux_compositions = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size', 'More sustainable materials']
df_compositions = pd.DataFrame()
# unique columns for all products
aux = []
cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size', 'More sustainable materials']
df_pattern = pd.DataFrame( columns=cols )
for i in range( len( data ) ):
    
    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id']+ '.html'
    page = requests.get( url, headers=headers )
    
    # Beautiful Soup object
    soup = BeautifulSoup( page.text, 'html.parser' )

    
    # ==================== color name =================================
    product_list = soup.find_all( 'a', class_='filter-option miniature' ) + soup.find_all( 'a', class_='filter-option miniature-active' )
    color_name = [p.get( 'data-color' ) for p in product_list]
    
    
    # product id
    product_id = [p.get( 'data-articlecode' ) for p in product_list]
    df_color = pd.DataFrame( [product_id, color_name] ).T
    df_color.columns = ['product_id', 'color_name']
    
    for j in range(len(df_color)):
        # API Requests
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j,'product_id'] + '.html'
        #print( 'Color: {}'.format( url ) )
        page = requests.get( url, headers=headers )
        # Beautiful Soup object
        soup = BeautifulSoup( page.text, 'html.parser' )
        # ================ Product Name ============================
        product_name = soup.find_all( 'h1', class_='primary product-item-headline' )
        product_name = product_name[0].get_text()
        
        # ================ Product Price ============================
        product_price = soup.find_all( 'div', class_='primary-row product-item-price' )
        product_price = re.findall( r'\d+\.?\d+', product_price[0].get_text())[0]    

        # ==================== composition =================================
        product_composition_list = soup.find_all( 'div', class_='pdp-description-list-item' )
        product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]
        
        # rename dataframe
        df_composition = pd.DataFrame( product_composition ).T
        df_composition.columns = df_composition.iloc[0]
        
        # delete first row
        df_composition = df_composition.iloc[1:].fillna( method='ffill' )

        #remove pocket ling, shell and lining
        df_composition['Composition'] = df_composition['Composition'].replace('Pocket lining:', '', regex = True )
        df_composition['Composition'] = df_composition['Composition'].replace('Shell:', '', regex = True )
        df_composition['Composition'] = df_composition['Composition'].replace('Lining:', '', regex = True )

        #print(df_composition)
        
        # garantee the same number of columns
        
        df_composition = pd.concat( [df_pattern, df_composition], axis=0 )

        #Rename columns
        df_composition.columns = ['product_id', 'composition', 'fit', 'product_safety', 'size', 'more_sustainable_materials']
        df_composition['product_name'] = product_name
        df_composition['product_price'] = product_price

        #Keep new columns if it shows up
        aux = aux + df_composition.columns.tolist()

        # merge data color + decomposition 
        
        df_composition = pd.merge( df_composition, df_color, how='left', on='product_id' )
        
        # all products
        df_compositions = pd.concat( [df_compositions, df_composition], axis=0 )

#print(df_composition)
# Join Showroom data + details
df_compositions['style_id'] = df_compositions['product_id'].apply( lambda x: x[:-3] )
df_compositions['color_id'] = df_compositions['product_id'].apply( lambda x: x[-3:] )
# scrapy datetime
df_compositions['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )
#df_compositions = df_compositions.drop_duplicates(subset=['style_id', 'product_id', 'color_id'])


In [48]:
df_compositions.head()

,product_id,composition,fit,product_safety,size,more_sustainable_materials,product_name,product_price,color_name,style_id,color_id,scrapy_datetime
0,0690449001,"Cotton 99%, Spandex 1%",Skinny fit,NaN,NaN,NaN,\n\t\t\t\t\t\t\t Skinny Jeans,16.99,Light denim blue/trashed,0690449,001,2021-10-24 11:53:43
0,0690449002,"Cotton 98%, Spandex 2%",Skinny fit,NaN,NaN,NaN,\n\t\t\t\t\t\t\t Skinny Jeans,14.99,Denim blue,0690449,002,2021-10-24 11:53:43
0,0690449006,Cotton 100%,Skinny fit,NaN,NaN,NaN,\n\t\t\t\t\t\t\t Skinny Jeans,7.99,Black/washed,0690449,006,2021-10-24 11:53:43
1,0690449006,"Cotton 98%, Spandex 2%",Skinny fit,NaN,NaN,NaN,\n\t\t\t\t\t\t\t Skinny Jeans,7.99,Black/washed,0690449,006,2021-10-24 11:53:43
0,0690449007,"Cotton 98%, Spandex 2%",Skinny fit,NaN,NaN,NaN,\n\t\t\t\t\t\t\t Skinny Jeans,14.99,Light denim blue,0690449,007,2021-10-24 11:53:43


# **3.0 Data Cleaning**    

In [140]:
# # product id
df_data = df_compositions.dropna(subset = ['product_id'])
# # product name
df_data['product_name'] = df_data['product_name'].str.replace('\n', '')
df_data['product_name'] = df_data['product_name'].str.replace('\t', '')
df_data['product_name'] = df_data['product_name'].str.replace('  ', '')
df_data['product_name'] = df_data['product_name'].str.replace(' ', '_')
df_data['product_name'] = df_data['product_name'].str.lower()

# #product_price
df_data['product_price'] = df_data['product_price'].astype( float )

# color name
df_data['color_name'] = df_data['color_name'].replace(' ', '_').str.lower()

# fit
df_data['fit'] = df_data['fit'].apply( lambda x: x.replace( ' ', '_' ).lower() if pd.notnull( x ) else x )

# size number
df_data['size_number'] = df_data['size'].apply( lambda x: re.search( '\d{3}cm', x ).group(0) if pd.notnull( x ) else x )
df_data['size_number'] = df_data['size_number'].apply( lambda x: re.search( '\d+', x).group(0) if pd.notnull( x ) else x )

# # # size model
df_data['size_model'] = df_data['size'].str.extract( '(\d+/\\d+)' )

# ================== composition =================
# break composition by comma
df1 = df_data['composition'].str.split( ',', expand=True ).reset_index(drop= True)
# cotton | polyester | elastano | elasterell
df_ref = pd.DataFrame( index=np.arange( len( data ) ),columns=['cotton','polyester', 'elastane', 'elasterell'] )
# ===========================cotton=====================
df_cotton_0 = df1.loc[df1[0].str.contains('Cotton', na=True),0]
df_cotton_0.name = 'cotton'

df_cotton_1 = df1.loc[df1[1].str.contains('Cotton', na=True),1]
df_cotton_1.name = 'cotton'

#combine
df_cotton = df_cotton_0.combine_first(df_cotton_1)

df_ref = pd.concat( [df_ref, df_cotton ], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]

#  ===================== polyester==========================
df_polyester_0 = df1.loc[df1[0].str.contains( 'Polyester', na=True ), 0]
df_polyester_0.name = 'polyester'

df_polyester_1 = df1.loc[df1[1].str.contains( 'Polyester', na=True ), 1]
df_polyester_1.name = 'polyester'

#combine
df_polyester = df_polyester_0.combine_first(df_polyester_1)
df_ref = pd.concat( [df_ref, df_polyester], axis=1 )
# df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]
# df_ref['polyester'] = df_ref['polyester'].fillna( 'Polyester 0%' )

# ================= elasterell ==============
df_elasterell = df1.loc[df1[1].str.contains( 'Elasterell', na=True ), 1]
df_elasterell.name = 'elasterell'
df_ref = pd.concat( [df_ref, df_elasterell], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]

#join of combine with product_id
df_aux = pd.concat([df_data['product_id'].reset_index(drop=True), df_ref], axis = 1)

# # format composition data
df_aux['cotton'] = df_aux['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['polyester'] = df_aux['polyester'].apply( lambda x: int( re.search( '\d+', x).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['elastane'] = df_aux['elastane'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['elasterell'] = df_aux['elasterell'].apply( lambda x: int(re.search('\d+',x ).group(0) ) / 100 if pd.notnull( x ) else x )

# # final join
df_aux = df_aux.groupby('product_id').max().reset_index().fillna(0)
df_data = pd.merge(df_data, df_aux, how = 'left', on = 'product_id')
 # Drop columns
df_data = df_data.drop( columns=['size', 'product_safety', 'composition'], axis=1 )
# # Drop duplicates
df_data = df_data.drop_duplicates()
df_data.shape

(116, 15)